## Chatbot por clasificador SVM, de proximidad entre expresiones

La funcionalidad de un chatbot por proximidad a una intención tambien es limitada, aunque más versátil que las anteriores según la calidad de las expresiones y la cantidad de clases de intenciones utilizadas para el entrenamiento.
Aunque en un modelo demostrativo es evidente que si se equivoca la intención enseguida aparece una respuesta inapropiada, su análisis resulta útil como introducción al uso de técnicas de aprendizaje automático con clasificadores de soporte vectorial o máquinas SVM

In [1]:
### #############################################################
### Chatbot por clasificador SVM de proximidad entre expresiones
### Jose Maria de Cuenca
### #############################################################

# Cargo librerías y funciones generales
import numpy as np # librería de cálculo matricial
import re # librería "regular expressions", para análisis de expresiones regulares
from collections import defaultdict # función para manejar listas de palabras con un diccinario
from sklearn.svm import SVC # función SVC de la librería de máquinas de soporte vectorial SVM de Scikit-Learn

In [2]:
# Cargo librería de procesamiento del lenguaje. Si ella o el paquete de lenguaje no está instalada debo hacerlo

import spacy # librería de procesado del lenguaje natural
# Si no está puedo buscarla en el interface de entorno de Anaconda, o mejor, cargar en linea de comandos con
# conda install -c conda-forge spacy

# Y tendré tambien que añadir las tablas y reglas para la funcionalidad de identificacion de lemas
# conda install -c conda-forge spacy-lookups-data


# Importo el paquete de lenguaje para la máquina. Opcionalmente podría descargarlo cada vez que deba usarlo (con la instrucción spacy.load("en_core_web_lg"))
# Opto por importarlo para reducir el tiempo de ejecución del código
import es_core_news_lg # Large model para ESpañol

# Si no tengo el paquete de lenguaje dará error: debo instalarlo mediante línea de comandos de Anaconda
# Puestos a instalar, lo mejor es cargar dos idiomas, el inglés, que usan la mayoría de ejemplos disponibles en la red. Y el de español para esta máquina. 
# Siempre debemos instalar los paquetes extendidos, porque incluyen la generación de vectores con las palabras
# python -m spacy download en_core_web_lg
# python -m spacy download es_core_news_lg

# Cargo el módulo de lenguaje natural del español. Puede tardar tiempo en función del tamaño del paquete
nlp = spacy.load('es_core_news_lg')

# Alternativamente, como paquete de carga directa
# nlp = es_core_news_lg.load()

In [3]:
# Defino una variable para almacenar la cadena de pregunta y respuesta con un formato que incluye un separador de guiones
contestacion = "Pregunta: {input}\nRespuesta: {output}\n" + "-"*100

In [4]:
# Análisis de intencion

# Defino dos listas con datos para entrenamiento, una con frases típicas y otra con la intención del interlocutor
# Ambas deben tener la misma longiud, y corresponderse ordenadamente las frases con la intención de las mismas

lista_frases_entrenamiento = [
    "¿Donde está su oficina?",
    "Necesito atención presencial personalizada",
    "Quiero verles, ¿puede darme una cita?",
    "Me gustaría tener una entrevista",
    "¿Puedo pasar a visitarles?",
    "¿Atienden al público en oficina?",
    "¿Por qué es tan cara el agua?",
    "El precio del agua es demasiado elevado",
    "El suministro de agua es carísimo",
    "¿Cuanto cuesta el agua?",
    "Creo que la tarifa de agua es abusiva",
    "La tasa de agua es muy cara",
    "¿Por qué pago tanto de agua?",
    "Creo que esta factura esta mal",
    "El recibo debe ser erróneo, es mayor que otras veces",
    "El cargo por agua en el banco es demasiado alto",
    "He pagado mucho más de lo normal por el agua esta vez",
    "¿Puede explicarme de donde sale el importe de un recibo tan alto?",
    "Quiero pagar el recibo de agua",
    "Deseo pagar esta factura de agua",
    "Voy a cerrar mis cuentas del banco ¿Cómo hago para pagar las facturas de agua?",
    "Quiero hacer un ingreso por este recibo",
    "¿Qué opciones hay para pagar aparte de la domiciliación bancaria?",
    "No quiero domiciliar los recibos en el banco",
    "Quiero darme de alta",
    "¿Qué hay que hacer para contratar el agua?",
    "¿Cómo puedo abonarme al servicio?",
    "Quiero transpasar el suministro de agua",
    "Quiero realizar un cambio de titular del contrato",
    "¿Qué hago para tener contrato de suministro?"
]

lista_intenciones = [
    "presencial",
    "presencial",
    "presencial",
    "presencial",
    "presencial",
    "presencial",
    "precio",
    "precio",
    "precio",
    "precio",
    "precio",
    "precio",
    "factura",
    "factura",
    "factura",
    "factura",
    "factura",
    "factura",
    "ingreso",
    "ingreso",
    "ingreso",
    "ingreso",
    "ingreso",
    "ingreso",
    "alta",
    "alta",
    "alta",
    "alta",
    "alta",
    "alta"
]

# Compruebo que las longitudes de ambas listas sean iguales
print('NUM FRASES:', len(lista_frases_entrenamiento))
print('NUM INTENCIONES: ', len(lista_intenciones))

NUM FRASES: 30
NUM INTENCIONES:  30


In [5]:
# OBTENGO EL NUMERO DE CLASES QUE USARA LA MAQUINA SVM

# Creo una variable de lista para el resumen de valores unicos de la lista de intenciones
resumen=[]

# Resumo la lista en otra resumida, con sus valores únicos
for intencion in lista_intenciones:
    if intencion not in resumen:
        resumen.append(intencion)
# Muestro el resumen de la lista de intenciones
print('LISTA RESUMEN: ', resumen)

# Calculo el numero de clases (intenciones de los interlocutores) que usaré en la clasificación
n_clases = len(resumen)

print('NUMERO DE CLASES PARA SVM: ', n_clases)

LISTA RESUMEN:  ['presencial', 'precio', 'factura', 'ingreso', 'alta']
NUMERO DE CLASES PARA SVM:  5


In [6]:
# CONVIERTO LAS FRASES DE ENTRENAMIENTO EN UNA MATRIZ DE VALORES

# Creo una matriz vacía llena de ceros para entremamiento, con dimensiones:
# filas = la longitud de la lista de frases
# columnas = la necesaria para caracterizar un texto (cualquiera) al procesarlo con la librería de análisis de lenguaje natural, que proporciona sus resultados como vectores de valores numéricos.
X_train = np.zeros((len(lista_frases_entrenamiento), 
              nlp('cualquier texto').vocab.vectors_length))

# Muestro las dimensiones de la nueva matriz
print('DIMENSIONES: ', X_train.shape)

# Coloco cada elemento de la lista (frase) en la matriz numérica, una vez procesado
for i, frase in enumerate(lista_frases_entrenamiento):
    # Examino cada frase extraída de la lista con el paquete de lenguaje nlp, generando un "documento" de Spacy
    # Un documento Spacy es una entidad compuesta por tokens o símbolos, creada a partir de la sucesión de los caracteres del texto procesado.
    doc = nlp(frase)
    # Obtengo el vector del documento Spacy, y lo coloco en la fila correspondiente a su frase dentro de la matriz.
    X_train[i, :] = doc.vector

# Muestro la matriz creada a partir de los vectores
print(X_train)

DIMENSIONES:  (30, 300)
[[-0.48439166  0.16172822  0.0197233  ...  1.18150675 -1.10268128
  -2.74171329]
 [-0.01730499 -0.58581495  0.56196499 ...  0.47120002 -0.92192996
  -1.31074262]
 [ 0.35004893 -0.92674106  0.21061188 ...  0.90740556 -0.28749284
  -2.12255001]
 ...
 [-0.81747168  1.39670372  1.3765167  ...  1.41383553  0.21690518
   0.18176667]
 [-0.49268624  1.59670877  0.04207873 ...  0.83498442  0.29082626
   0.17766005]
 [-0.48325783 -0.5238719   0.97486329 ...  0.37917364 -0.46799108
  -2.91752791]]


In [7]:
# CREO EL MODELO CLASIFICADOR DE FRASES PARA ANALIZAR SU INTENCIÓN

# Usaremos una máquina de vectores de soporte (SVM), que clasificará las frases por su proximidad espacial
# El espacio a considerar será de tantas dimensiones como las columnas de la matriz usada para entrenar el modelo

# Defino el modelo como una máquina SVM utiliando la función SVC de la librería SVM de Scikit-Learn
mod_clasificador = SVC(C=1, gamma="auto", probability=True, class_weight='balanced') # C=factor de penalización, gamma=coeficiente de ajuste, auto usa 1/n características; probability=usa validación cruzada, class_weight ajusta pesos proporcionalmente a los elementos de cada clase

# Entreno el modelo de clasificador SVM
# Durante el entrenamiento, el modelo busca los hiperplanos que separan óptimamente las zonas que corresponden a cada etiqueta (intención)
# Los datos son la matriz de vectores con las frases caracterizadas numéricamente, y la lista de etiquetas que definen la intención de cada una
mod_clasificador.fit(X_train, lista_intenciones)

SVC(C=1, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [8]:
# Defino una función que analiza la intención del interlocutor utilizando el modelo clasificador creado
def get_intencion_svm(cuestion):
    # Convierto la cuestion en un nuevo "documento" de Spacy
    doc = nlp(cuestion)
    # Obtengo la máxima probabilidad entre las que corresponden a cada clase segun el modelo de clasificación para esta cuestion
    # Si el conjunto de datos es reducido, la funcion precit_proba de un modelo SVM suministrada por Scikit-Learn no es muy fiable
    max_proba = max(mod_clasificador.predict_proba([doc.vector])[0])
    # Si no alcanza un valor suficiente (mayor al que le corresponde aleatoriamente (1/n clases), responerá por defecto)
    if(max_proba <= (1/n_clases)):
        return('defecto')
    else:
        return(mod_clasificador.predict([doc.vector])[0])


In [9]:
# Pruebo la funcion de analisis de intenciones

# Pruebo la extraccion de la intencion principal con una frase que no tiene nada que ver
intencion_principal = get_intencion_svm('Te invito a cenar esta noche')
# Muestro el resultado de manera ordenada
print('INTENCION PRINCIPAL: ', intencion_principal)

# Pruebo con otra frase
print('INTENCION PRINCIPAL: ', get_intencion_svm('La cena fué demasiado cara'))

INTENCION PRINCIPAL:  presencial
INTENCION PRINCIPAL:  precio


In [10]:
# Creo la lista con la respuesta más adecuada a cada intencion del interlocutor
lista_respuestas = {
    "presencial":"Puede visitarnos en la Casa Consistorial, en la plaza Mayor número 1.",
    "precio":"El agua tiene muchos costes asociados que se incluyen en la tarifa, que es una de las más baratas.",
    "factura":"Si su última factura fué muy superior a las anteriores, por favor verifique si la lectura de su contador es correcta, y que no tiene fugas en su instalación.",
    "ingreso":"La forma de pago más cómoda, barata y fiable es la domiciliación bancaria. Si no le convence, puede pasar con su factura por una oficina de correos.",
    "alta":"Debe traer o enviar electrónicamente la documentación siguiente....",
    
    "defecto":"Lo siento, no logré entender su pregunta. Estaré encantado de intentar ayudarle si la replantea."
}


# Compruebo que la lista de posibles respuestas se corresponde al número de clases o intenciones, más la respuesta por defecto
print('NÚMERO DE CLASES: ', n_clases)
print('NÚMERO DE RESPUESTAS: ', len(lista_respuestas))

NÚMERO DE CLASES:  5
NÚMERO DE RESPUESTAS:  6


In [11]:
# Defino una funcion que establece la correspondencia entre la intención del interlocutor y la respuesta a mostrar
def chatbot_svm(cuestion):
    # Determino la respuesta que corresponde a la clasificación de intenciones, para la cuestion planteada
    respuesta = lista_respuestas.get(get_intencion_svm(cuestion), lista_respuestas["defecto"])
    # Devuelvo el resultado en forma de cadena de contestación, apropiada para imprimir
    return(contestacion.format(input=cuestion, output=respuesta))

In [27]:
# Ejemplos de diálogo:

print(chatbot_svm("Quiero contratar el alta del suministro"))

print(chatbot_svm("¿Donde tienen su oficina?"))

print(chatbot_svm("Debe haber un error en su recibo"))

print(chatbot_svm("El agua es un robo"))

print(chatbot_svm("¿Cuales son sus formas de pago?"))

print(chatbot_svm("No me convence tratar con una máquina"))

print(chatbot_svm("¿Eres realmente inteligente?"))


Pregunta: Quiero contratar el alta del suministro
Respuesta: Debe traer o enviar electrónicamente la documentación siguiente....
----------------------------------------------------------------------------------------------------
Pregunta: ¿Donde tienen su oficina?
Respuesta: Puede visitarnos en la Casa Consistorial, en la plaza Mayor número 1.
----------------------------------------------------------------------------------------------------
Pregunta: Debe haber un error en su recibo
Respuesta: Si su última factura fué muy superior a las anteriores, por favor verifique si la lectura de su contador es correcta, y que no tiene fugas en su instalación.
----------------------------------------------------------------------------------------------------
Pregunta: El agua es un robo
Respuesta: El agua tiene muchos costes asociados que se incluyen en la tarifa, que es una de las más baratas.
----------------------------------------------------------------------------------------------------